In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from preprocessing import load_events, load_bing_results, check_counts_and_merge, load_gni_class, load_gni_region,load_population, load_gdp, load_if_in_country_or_lang, load_views
%load_ext autoreload
%autoreload 2

## load df

In [2]:
df = load_events()
df_news = load_bing_results()
df_crawled = check_counts_and_merge(df, df_news)
# show article counts
df_crawled.head()

Loading de
LOAD FROM scraping/results/de_bingnews.json, Articles now: 2304
LOAD FROM scraping/results//de_bingnews_missing.json, Articles now: 2376
Loading en
LOAD FROM scraping/results/en_bingnews.json, Articles now: 5262
LOAD FROM scraping/results/en_bingnews_rest.json, Articles now: 7761
LOAD FROM scraping/results/en_bingnews_missing.json, Articles now: 7761
Loading it
LOAD FROM scraping/results/it_bingnews.json, Articles now: 2297
LOAD FROM scraping/results//it_bingnews_missing.json, Articles now: 2358
Loading es
LOAD FROM scraping/results//es_bingnews_es-ES.json, Articles now: 3253
LOAD FROM scraping/results//es_bingnews_es-ES_missing.json, Articles now: 3448
Loading es
LOAD FROM scraping/results/es_bingnews_es-ES_corrected.json, Articles now: 552
de 2376 2502
it 2358 2470
es 3448 3587
en 7761 8021
total 16580 15943
With duplicates, but dropped na from bing 16580
Without duplicates 14198
de 1994
it 1804
es 3051
en 7349


,event_id,pagetitle,event_date,economic_region,country,continent,cat,categories,views_7_sum,views_before_mean,...,factor,page_creation,edits_7_sum,edits_before_min,edits_before_mean,edits_before_max,edits_before_median,edits_before_sum,code,bing_hits
0,Q100021,Pakistan International Airlines Flight 661,2016-12-07,Global South,Pakistan,Asia,disaster,"['Category:2016 disasters in Pakistan', 'Categ...",248274,0.000000,...,NaN,2016-12-07T12:37:39Z,518,0,0.000000,0,0.0,0,en,11
1,Q100231328,Gretchen Whitmer kidnapping plot,2020-10-08,Global North,United States,North America,disaster,"['Category:2020 crimes in the United States', ...",74144,0.000000,...,NaN,2020-10-08T17:29:02Z,23,0,0.000000,0,0.0,0,en,6670
2,Q100256872,Killing of Babu Lal Vaishnav,2020-10-08,Global South,India,Asia,disaster,"['Category:2020 crimes in India', 'Category:20...",427,0.000000,...,after,2020-10-10T09:08:38Z,39,0,0.000000,0,0.0,0,en,33
3,Q100279235,Miss Vietnam 2020,2020-11-20,Global South,Vietnam,Asia,culture,"['Category:2020 beauty pageants', 'Category:Be...",1444,49.285714,...,before,2020-11-12T17:26:31Z,18,0,0.666667,3,0.0,4,en,44300
4,Q100288161,2020 Ganja missile attacks,2020-10-11,Global South,Azerbaijan,Asia,disaster,"['Category:2020 in Azerbaijan', 'Category:Arti...",7410,0.000000,...,after,2020-10-15T10:07:12Z,44,0,0.000000,0,0.0,0,en,237000


## load gni region and regional class

In [3]:
df_crawled, df_missing_class = load_gni_class(df_crawled)
df_crawled, df_missing_region = load_gni_region(df_crawled)

Lost Events for class: 15
Lost Events for region:  0


In [14]:
df_missing_class.country.unique()

array(['Falkland Islands', 'Faeroe Islands', 'Cook Islands'], dtype=object)

## load population

In [5]:
df_crawled, df_missing_pop = load_population(df_crawled)

Lost Events for population:  0


# Merge with GDP

In [6]:
df_crawled, df_missing_gdp = load_gdp(df_crawled)

Lost Events for gdp:  3


In [15]:
df_missing_gdp.country.unique()

array(['Gibraltar'], dtype=object)

# compute lang and code assignment

In [8]:
df_crawled = load_if_in_country_or_lang(df_crawled)

# Store resulting dataframe, then change manually which events were not found

Note that these are commented out because we already did that once. the manually processed df is fine from here on out.

In [9]:
#df_crawled.to_csv(f'events/new/processed.csv')

In [16]:
#df_manually_processed = pd.read_csv('events/processed_manually.csv')

In [11]:
# some fixes for some missing events
#pd.concat([df_manually_processed, df_crawled[~df_crawled.event_id.isin(df_manually_processed.event_id)]]).to_csv(
# 'events/new/processed_manually.csv', index=False)

# Merge with views

In [20]:
from preprocessing import compute_view_baseline
from preprocessing import replace_country_names

In [24]:
df_new = pd.read_csv('events/new/processed_manually.csv')
df_new.event_date = pd.to_datetime(df_new.event_date)
df_new['date_month'] = df_new.event_date.apply(lambda d: pd.to_datetime(f'{d.year}-{d.month:02d}'))
df_new.head()

,event_id,pagetitle,event_date,economic_region,country,continent,cat,categories,views_7_sum,views_before_mean,...,bing_hits,gni_class,gni_region,population,Country/Area,Year,GDP_pc,GDP,oecd,date_month
0,Q100021,Pakistan International Airlines Flight 661,2016-12-07,Global South,Pakistan,Asia,disaster,"['Category:2016 disasters in Pakistan', 'Categ...",248274,0.000000,...,11,LM,South Asia,203631356,Pakistan,2016,1362.860008,2.775210e+11,False,2016-12-01
1,Q100231328,Gretchen Whitmer kidnapping plot,2020-10-08,Global North,United States,North America,disaster,"['Category:2020 crimes in the United States', ...",74144,0.000000,...,6670,H,North America,329484123,United States,2020,63122.594145,2.089375e+13,True,2020-10-01
2,Q100256872,Killing of Babu Lal Vaishnav,2020-10-08,Global South,India,Asia,disaster,"['Category:2020 crimes in India', 'Category:20...",427,0.000000,...,33,LM,South Asia,1380004385,India,2020,1930.971218,2.664749e+12,False,2020-10-01
3,Q100279235,Miss Vietnam 2020,2020-11-20,Global South,Vietnam,Asia,culture,"['Category:2020 beauty pageants', 'Category:Be...",1444,49.285714,...,44300,LM,East Asia & Pacific,97338583,Vietnam,2020,2785.724225,2.711584e+11,False,2020-11-01
4,Q100288161,2020 Ganja missile attacks,2020-10-11,Global South,Azerbaijan,Asia,disaster,"['Category:2020 in Azerbaijan', 'Category:Arti...",7410,0.000000,...,237000,UM,Europe & Central Asia,10093121,Azerbaijan,2020,4202.232239,4.260718e+10,False,2020-10-01


In [45]:
# this takes some time!
df_views = load_views()
df_views_rel, min_date = compute_view_baseline(df_views, start_date_views='2015-05-01', months_before=5,
                                               func_agg=np.median)

/home/ruptho/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/ruptho/.local/lib/python3.10/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [46]:
df_views_rel = df_views_rel[['code', 'date_month', 'country', 'views_baseline']]
#df_views_rel = replace_country_names(df_views_rel)

In [66]:
df_views_rel = replace_country_names(df_views_rel)
df_view_merged = df_new.merge(df_views_rel, left_on=['code', 'country', 'date_month'],
                              right_on=['code', 'country', 'date_month'], how='left')
df_view_merged = df_view_merged[df_view_merged.date_month >= min_date]

In [67]:
# check if results are valid
countries_na = df_view_merged[pd.isna(df_view_merged.views_baseline)].country.unique()
countries_not_na = df_view_merged[~pd.isna(df_view_merged.views_baseline)].country.unique()
df_countries_partial_data = df_view_merged[df_view_merged.country.isin(countries_na) & df_view_merged.country.isin(countries_not_na)][
    ['code', 'country', 'event_date', 'views_baseline']]
df_countries_no_data = df_view_merged[df_view_merged.country.isin(countries_na) & ~df_view_merged.country.isin(countries_not_na)][
    ['code', 'country', 'event_date', 'views_baseline']]
# YES THEY ARE! some countries don't have pageviews for a specific timespan...
df_countries_partial_data

array(['Namibia', 'Tuvalu', 'North Korea', 'Solomon Islands', 'Tonga',
       'Nauru'], dtype=object)

In [102]:
# ... and we just fill them with 0s here!
# in theory. we could also change how we handle this for these small-resource countries during data retrieval. but with median, this should be fine rn.
df_view_merged.views_baseline.fillna(0, inplace=True)

In [107]:
# so let's save this and use it now!
df_view_merged.to_csv('events/new/processed_manually_with_wikiviews.csv', index=False)